<a href="https://colab.research.google.com/github/sciyg/D-Wave/blob/main/%E9%87%8F%E5%AD%90%E3%82%A2%E3%83%8B%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0%E3%81%A7%E3%82%B7%E3%83%95%E3%83%88%E8%A1%A8%E4%BD%9C%E6%88%90_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 量子アニーリングでシフト表作成

以下のようなシフト表で、自動的に出勤日を決定できるようにします。

0が休日で、1が出勤日とします。

In [ ]:
# データをカレンダー表示する関数を作成

import numpy as np
import pandas as pd
weeks = ['日', '月', '火', '水', '木', '金', '土']
persons = ['A', 'B', 'C', 'D']

# 列名
def column_name(d):
    return f"{d + 1}({weeks[d % 7]})"

# 行名
def index_name(p):
    return persons[p]

# pandasでカレンダー表示する関数
def get_calendar(data):
  data_df = pd.DataFrame(data)
  data_df = data_df.astype('int')
  data_df = data_df.rename(columns=column_name, index=index_name)

  data_df = pd.concat([data_df,pd.DataFrame(data_df.sum(axis=0),columns=['出勤者数']).T])
  data_df = pd.concat([data_df,pd.DataFrame(data_df.sum(axis=1),columns=['出勤日数'])],axis=1)  
  return data_df

In [ ]:
# データが空のシフト表を表示
empty_data = np.zeros((4,30))
get_calendar(empty_data)

,1(日),2(月),3(火),4(水),5(木),6(金),7(土),8(日),9(月),10(火),11(水),12(木),13(金),14(土),15(日),16(月),17(火),18(水),19(木),20(金),21(土),22(日),23(月),24(火),25(水),26(木),27(金),28(土),29(日),30(月),出勤日数
A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
出勤者数,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


とりあえず最初に、**1日に二人が出勤する**、という制約条件だけで作ってみます。



## 前提条件

* 一ヶ月を30日とします
* 1日を日曜日とします
* 担当者を４人（A,B,C,D)とします

## 定式化

### 決定変数

担当者 $p\in \{0, 1, 2, 3\}$とし、担当者A（$p = 0$）・担当者B（$p = 1$）・担当者C（$p = 2$）・担当者D（$p = 3$）とします。

日にち  $d \in \{0, \cdots, 29\}$ とし、1日（$d = 0$）、...、30日（$d = 29$）とします。

担当者$p$が日にち$d$に出勤する状態を、バイナリ変数 $q_{p,d}$ を用いて表現することにします。

* 例1 Aが1日に出勤の時、$q_{0,0} = 1$
* 例2 Bが10日に休日の時、$q_{1,9} = 0$


|担当者 \ 日| $0$ | $1$ | ... |２９|  
|:---:|:---:|:---:|:---:|:---:|
|$0$| $q_{0,0}$ | $q_{0,1}$ | ... | $q_{0,29}$|
|$1$| $q_{1,0}$ | $q_{1,1}$ | ... | $q_{1,29}$|
|$２$| $q_{2,0}$ | $q_{2,1}$ | ... | $q_{2,29}$|
|$3$| $q_{3,0}$ | $q_{3,1}$ | ... | $q_{3,29}$|

### 制約条件

１日に必ず二人が出勤するように、制約条件を設定します。

$$
\sum_{p=0}^{Np -1}q_{p,d} = 2
$$

※ $Np$ = 4

### 目的関数

各担当者の出勤数の合計が同じになるように目的関数を作ります。
この値（エネルギーと呼ばれる）が最小になるように、イジングマシンが最適化してくれます。

$$
f = (担当者Aと担当者Bの出勤数の差分)^2 + (担当者Aと担当者Cの出勤数の差分)^2 + (担当者Aと担当者Dの出勤数の差分)^2 + (担当者Bと担当者Cの出勤数の差分)^2 + (担当者Bと担当者Dの出勤数の差分)^2+ (担当者Cと担当者Dの出勤数の差分)^2
$$

$$
= \left(\sum_{d=0}^{Nd-1}q_{d,0} - \sum_{d=0}^{Nd-1}q_{d,1}\right)^2 + \left(\sum_{d=0}^{Nd-1}q_{d,0} - \sum_{d=0}^{Nd-1}q_{d,2}\right)^2+ \left(\sum_{d=0}^{Nd-1}q_{d,0} - \sum_{d=0}^{Nd-1}q_{d,3}\right)^2+ \left(\sum_{d=0}^{Nd-1}q_{d,1} - \sum_{d=0}^{Nd-1}q_{d,2}\right)^2+ \left(\sum_{d=0}^{Nd-1}q_{d,1} - \sum_{d=0}^{Nd-1}q_{d,3}\right)^2+ \left(\sum_{d=0}^{Nd-1}q_{d,2} - \sum_{d=0}^{Nd-1}q_{d,3}\right)^2
$$

※ $Nd$ = 30


## 実装

まず、Amplify SDKを使えるようにします。

In [ ]:
!pip install amplify

     |████████████████████████████████| 7.1 MB 5.0 MB/s 


In [ ]:
from amplify import gen_symbols, BinaryPoly, Solver, decode_solution, sum_poly
from amplify.client import FixstarsClient
from amplify.constraint import equal_to, penalty

### 決定変数

バイナリー（0または1の値を取りうる）の決定変数を (Np x Nd) の配列の形式で定義します

In [ ]:
# 担当者の数
Np = 4

# 月の日数
Nd = 30

q = gen_symbols(BinaryPoly, Np, Nd)

In [ ]:
q

[[  q_0,   q_1,   q_2,   q_3,   q_4,   q_5,   q_6,   q_7,   q_8,   q_9, 
   q_10,  q_11,  q_12,  q_13,  q_14,  q_15,  q_16,  q_17,  q_18,  q_19, 
   q_20,  q_21,  q_22,  q_23,  q_24,  q_25,  q_26,  q_27,  q_28,  q_29],
 [ q_30,  q_31,  q_32,  q_33,  q_34,  q_35,  q_36,  q_37,  q_38,  q_39, 
   q_40,  q_41,  q_42,  q_43,  q_44,  q_45,  q_46,  q_47,  q_48,  q_49, 
   q_50,  q_51,  q_52,  q_53,  q_54,  q_55,  q_56,  q_57,  q_58,  q_59],
 [ q_60,  q_61,  q_62,  q_63,  q_64,  q_65,  q_66,  q_67,  q_68,  q_69, 
   q_70,  q_71,  q_72,  q_73,  q_74,  q_75,  q_76,  q_77,  q_78,  q_79, 
   q_80,  q_81,  q_82,  q_83,  q_84,  q_85,  q_86,  q_87,  q_88,  q_89],
 [ q_90,  q_91,  q_92,  q_93,  q_94,  q_95,  q_96,  q_97,  q_98,  q_99, 
  q_100, q_101, q_102, q_103, q_104, q_105, q_106, q_107, q_108, q_109, 
  q_110, q_111, q_112, q_113, q_114, q_115, q_116, q_117, q_118, q_119]]

### 制約条件

１日に二人の 制約を作ります。

In [ ]:
day_constraints =  sum(
    [equal_to(sum_poly(Np, lambda p: q[p][d]), 2) for d in range(Nd)]
)

In [ ]:
day_constraints

[(q_0 + q_30 + q_60 + q_90 == 2, 1), (q_1 + q_31 + q_61 + q_91 == 2, 1), (q_2 + q_32 + q_62 + q_92 == 2, 1), (q_3 + q_33 + q_63 + q_93 == 2, 1), (q_4 + q_34 + q_64 + q_94 == 2, 1), (q_5 + q_35 + q_65 + q_95 == 2, 1), (q_6 + q_36 + q_66 + q_96 == 2, 1), (q_7 + q_37 + q_67 + q_97 == 2, 1), (q_8 + q_38 + q_68 + q_98 == 2, 1), (q_9 + q_39 + q_69 + q_99 == 2, 1), (q_10 + q_40 + q_70 + q_100 == 2, 1), (q_11 + q_41 + q_71 + q_101 == 2, 1), (q_12 + q_42 + q_72 + q_102 == 2, 1), (q_13 + q_43 + q_73 + q_103 == 2, 1), (q_14 + q_44 + q_74 + q_104 == 2, 1), (q_15 + q_45 + q_75 + q_105 == 2, 1), (q_16 + q_46 + q_76 + q_106 == 2, 1), (q_17 + q_47 + q_77 + q_107 == 2, 1), (q_18 + q_48 + q_78 + q_108 == 2, 1), (q_19 + q_49 + q_79 + q_109 == 2, 1), (q_20 + q_50 + q_80 + q_110 == 2, 1), (q_21 + q_51 + q_81 + q_111 == 2, 1), (q_22 + q_52 + q_82 + q_112 == 2, 1), (q_23 + q_53 + q_83 + q_113 == 2, 1), (q_24 + q_54 + q_84 + q_114 == 2, 1), (q_25 + q_55 + q_85 + q_115 == 2, 1), (q_26 + q_56 + q_86 + q_116 == 

### 目的関数

In [ ]:
cost = (sum([q[0][d] for d in range(Nd)]) - sum([q[1][d] for d in range(Nd)]))**2 \
     + (sum([q[0][d] for d in range(Nd)]) - sum([q[2][d] for d in range(Nd)]))**2 \
     + (sum([q[0][d] for d in range(Nd)]) - sum([q[3][d] for d in range(Nd)]))**2 \
     + (sum([q[1][d] for d in range(Nd)]) - sum([q[2][d] for d in range(Nd)]))**2 \
     + (sum([q[1][d] for d in range(Nd)]) - sum([q[3][d] for d in range(Nd)]))**2 \
     + (sum([q[2][d] for d in range(Nd)]) - sum([q[3][d] for d in range(Nd)]))**2  

In [ ]:
cost

6 q_0 q_1 + 6 q_0 q_2 + 6 q_0 q_3 + 6 q_0 q_4 + 6 q_0 q_5 + 6 q_0 q_6 + 6 q_0 q_7 + 6 q_0 q_8 + 6 q_0 q_9 + 6 q_0 q_10 + 6 q_0 q_11 + 6 q_0 q_12 + 6 q_0 q_13 + 6 q_0 q_14 + 6 q_0 q_15 + 6 q_0 q_16 + 6 q_0 q_17 + 6 q_0 q_18 + 6 q_0 q_19 + 6 q_0 q_20 + 6 q_0 q_21 + 6 q_0 q_22 + 6 q_0 q_23 + 6 q_0 q_24 + 6 q_0 q_25 + 6 q_0 q_26 + 6 q_0 q_27 + 6 q_0 q_28 + 6 q_0 q_29 - 2 q_0 q_30 - 2 q_0 q_31 - 2 q_0 q_32 - 2 q_0 q_33 - 2 q_0 q_34 - 2 q_0 q_35 - 2 q_0 q_36 - 2 q_0 q_37 - 2 q_0 q_38 - 2 q_0 q_39 - 2 q_0 q_40 - 2 q_0 q_41 - 2 q_0 q_42 - 2 q_0 q_43 - 2 q_0 q_44 - 2 q_0 q_45 - 2 q_0 q_46 - 2 q_0 q_47 - 2 q_0 q_48 - 2 q_0 q_49 - 2 q_0 q_50 - 2 q_0 q_51 - 2 q_0 q_52 - 2 q_0 q_53 - 2 q_0 q_54 - 2 q_0 q_55 - 2 q_0 q_56 - 2 q_0 q_57 - 2 q_0 q_58 - 2 q_0 q_59 - 2 q_0 q_60 - 2 q_0 q_61 - 2 q_0 q_62 - 2 q_0 q_63 - 2 q_0 q_64 - 2 q_0 q_65 - 2 q_0 q_66 - 2 q_0 q_67 - 2 q_0 q_68 - 2 q_0 q_69 - 2 q_0 q_70 - 2 q_0 q_71 - 2 q_0 q_72 - 2 q_0 q_73 - 2 q_0 q_74 - 2 q_0 q_75 - 2 q_0 q_76 - 2 q_0 q_77 - 2 q_0 q_

上記だと、人数変更したときに不便なため、一般化してみました。**itertools.combinations** でリストから組み合わせを作成できます。

In [ ]:
import itertools

In [ ]:
[t for t in itertools.combinations([p for p in range(Np)], 2)]

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]

In [ ]:
c = itertools.combinations([p for p in range(Np)], 2)
cost = sum_poly([(sum_poly(Nd, lambda d:q[tp[0]][d]) - sum_poly(Nd, lambda d:q[tp[1]][d]))**2 for tp in c])

In [ ]:
cost

6 q_0 q_1 + 6 q_0 q_2 + 6 q_0 q_3 + 6 q_0 q_4 + 6 q_0 q_5 + 6 q_0 q_6 + 6 q_0 q_7 + 6 q_0 q_8 + 6 q_0 q_9 + 6 q_0 q_10 + 6 q_0 q_11 + 6 q_0 q_12 + 6 q_0 q_13 + 6 q_0 q_14 + 6 q_0 q_15 + 6 q_0 q_16 + 6 q_0 q_17 + 6 q_0 q_18 + 6 q_0 q_19 + 6 q_0 q_20 + 6 q_0 q_21 + 6 q_0 q_22 + 6 q_0 q_23 + 6 q_0 q_24 + 6 q_0 q_25 + 6 q_0 q_26 + 6 q_0 q_27 + 6 q_0 q_28 + 6 q_0 q_29 - 2 q_0 q_30 - 2 q_0 q_31 - 2 q_0 q_32 - 2 q_0 q_33 - 2 q_0 q_34 - 2 q_0 q_35 - 2 q_0 q_36 - 2 q_0 q_37 - 2 q_0 q_38 - 2 q_0 q_39 - 2 q_0 q_40 - 2 q_0 q_41 - 2 q_0 q_42 - 2 q_0 q_43 - 2 q_0 q_44 - 2 q_0 q_45 - 2 q_0 q_46 - 2 q_0 q_47 - 2 q_0 q_48 - 2 q_0 q_49 - 2 q_0 q_50 - 2 q_0 q_51 - 2 q_0 q_52 - 2 q_0 q_53 - 2 q_0 q_54 - 2 q_0 q_55 - 2 q_0 q_56 - 2 q_0 q_57 - 2 q_0 q_58 - 2 q_0 q_59 - 2 q_0 q_60 - 2 q_0 q_61 - 2 q_0 q_62 - 2 q_0 q_63 - 2 q_0 q_64 - 2 q_0 q_65 - 2 q_0 q_66 - 2 q_0 q_67 - 2 q_0 q_68 - 2 q_0 q_69 - 2 q_0 q_70 - 2 q_0 q_71 - 2 q_0 q_72 - 2 q_0 q_73 - 2 q_0 q_74 - 2 q_0 q_75 - 2 q_0 q_76 - 2 q_0 q_77 - 2 q_0 q_

### イジングマシンの実行

イジングマシンのクライアントを `FixstarsClient` に設定し、さらにソルバーを作成して、以下のように問題を解きます。

In [ ]:
# クライアントを設定
client = FixstarsClient()
client.parameters.timeout = 1000  # タイムアウト1秒
client.parameters.outputs.duplicate = True # 同じエネルギー値の解を列挙
client.token = "***************"  # Amplify AEのアクセストークンを入力してください

# ソルバーを設定
solver = Solver(client)

目的関数と制約条件を結合します。  
数値の10は、制約条件の強さの係数です。10は適当に決めました。

In [ ]:
model = cost + 10 * day_constraints

問題をときます。

解けない場合、例外を発生させます。

In [ ]:
result = solver.solve(model)
if len(result) == 0:
    raise RuntimeError("条件を満たす解は見つかりませんでした。")

In [ ]:
# 解の数
len(result)

7145

In [ ]:
# 解0の結果
r0 = decode_solution(q, result[0].values)
print(f"f={result[0].energy}") # 最適化関数の値
get_calendar(r0)

f=0.0


,1(日),2(月),3(火),4(水),5(木),6(金),7(土),8(日),9(月),10(火),11(水),12(木),13(金),14(土),15(日),16(月),17(火),18(水),19(木),20(金),21(土),22(日),23(月),24(火),25(水),26(木),27(金),28(土),29(日),30(月),出勤日数
A,0,0,0,0,0,1,1,1,0,0,1,1,1,0,1,1,0,0,1,1,0,1,0,1,0,0,1,1,1,0,15
B,1,1,1,1,1,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,1,1,1,1,0,0,1,0,0,15
C,1,1,0,1,1,0,0,1,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,1,15
D,0,0,1,0,0,0,1,0,1,1,0,0,0,1,0,1,1,1,1,1,1,0,1,0,1,1,0,0,0,1,15
出勤者数,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,60


In [ ]:
# 解1の結果
r1 = decode_solution(q, result[1].values)
get_calendar(r1)

,1(日),2(月),3(火),4(水),5(木),6(金),7(土),8(日),9(月),10(火),11(水),12(木),13(金),14(土),15(日),16(月),17(火),18(水),19(木),20(金),21(土),22(日),23(月),24(火),25(水),26(木),27(金),28(土),29(日),30(月),出勤日数
A,0,1,1,1,1,0,1,1,0,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,1,1,15
B,0,0,0,0,0,1,1,0,0,1,1,0,1,1,0,0,1,1,0,0,1,1,1,1,1,0,1,0,1,0,15
C,1,0,1,0,1,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,1,1,0,0,1,1,0,1,0,1,15
D,1,1,0,1,0,0,0,1,1,0,0,1,0,1,1,1,1,1,0,1,0,0,1,1,0,1,0,0,0,0,15
出勤者数,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,60


次に、以下の条件を追加します。

* 連続６勤務以上は禁止
* 担当者A：どの日でもOK
* 担当者B：土日のみ出勤可
* 担当者C：土日以外のみ出勤可
* 担当者D：1日〜5日は休み

## 連続６勤務以上は禁止の制約

In [ ]:
# 連続6勤務だと、q[p][d] * q[p][d+1] * q[p][d+2] * q[p][d+3]  * q[p][d+4] * q[p][d+5] ≠ 0 になる
# penalty関数で0とする
overwork_constraints =  sum(
    [penalty(q[p][d] * q[p][d+1] * q[p][d+2] * q[p][d+3]  * q[p][d+4] * q[p][d+5]) for d in range(Nd - 5) for p in range(Np)]
)

In [ ]:
overwork_constraints

[(q_0 q_1 q_2 q_3 q_4 q_5 == 0, 1), (q_30 q_31 q_32 q_33 q_34 q_35 == 0, 1), (q_60 q_61 q_62 q_63 q_64 q_65 == 0, 1), (q_90 q_91 q_92 q_93 q_94 q_95 == 0, 1), (q_1 q_2 q_3 q_4 q_5 q_6 == 0, 1), (q_31 q_32 q_33 q_34 q_35 q_36 == 0, 1), (q_61 q_62 q_63 q_64 q_65 q_66 == 0, 1), (q_91 q_92 q_93 q_94 q_95 q_96 == 0, 1), (q_2 q_3 q_4 q_5 q_6 q_7 == 0, 1), (q_32 q_33 q_34 q_35 q_36 q_37 == 0, 1), (q_62 q_63 q_64 q_65 q_66 q_67 == 0, 1), (q_92 q_93 q_94 q_95 q_96 q_97 == 0, 1), (q_3 q_4 q_5 q_6 q_7 q_8 == 0, 1), (q_33 q_34 q_35 q_36 q_37 q_38 == 0, 1), (q_63 q_64 q_65 q_66 q_67 q_68 == 0, 1), (q_93 q_94 q_95 q_96 q_97 q_98 == 0, 1), (q_4 q_5 q_6 q_7 q_8 q_9 == 0, 1), (q_34 q_35 q_36 q_37 q_38 q_39 == 0, 1), (q_64 q_65 q_66 q_67 q_68 q_69 == 0, 1), (q_94 q_95 q_96 q_97 q_98 q_99 == 0, 1), (q_5 q_6 q_7 q_8 q_9 q_10 == 0, 1), (q_35 q_36 q_37 q_38 q_39 q_40 == 0, 1), (q_65 q_66 q_67 q_68 q_69 q_70 == 0, 1), (q_95 q_96 q_97 q_98 q_99 q_100 == 0, 1), (q_6 q_7 q_8 q_9 q_10 q_11 == 0, 1), (q_36 q_37 q

## 担当者Bの制約

In [ ]:
# 土日のみの配列
[d for d in range(Nd) if (d % 7 == 0 or (d + 1) % 7 == 0)]

[0, 6, 7, 13, 14, 20, 21, 27, 28]

In [ ]:
# 土日以外の配列
[d for d in range(Nd) if (d % 7 != 0 and (d + 1) % 7 != 0)]

[1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29]

In [ ]:
# B:土日のみ出勤する制約条件
# 土日以外は、penalty関数で0とする
b_constraints =  sum(
    [penalty(q[1][d]) for d in range(Nd) if (d % 7 != 0 and (d + 1) % 7 != 0)]
)

In [ ]:
b_constraints

[(q_31 == 0, 1), (q_32 == 0, 1), (q_33 == 0, 1), (q_34 == 0, 1), (q_35 == 0, 1), (q_38 == 0, 1), (q_39 == 0, 1), (q_40 == 0, 1), (q_41 == 0, 1), (q_42 == 0, 1), (q_45 == 0, 1), (q_46 == 0, 1), (q_47 == 0, 1), (q_48 == 0, 1), (q_49 == 0, 1), (q_52 == 0, 1), (q_53 == 0, 1), (q_54 == 0, 1), (q_55 == 0, 1), (q_56 == 0, 1), (q_59 == 0, 1)]

## 担当者Cの制約

In [ ]:
# C:土日以外のみ出勤する制約条件
# 土日はequal_to関数で0とする、equal_to(q[2][d], 0)はpenalty(q[2][d])と同じ
c_constraints =  sum(
    [equal_to(q[2][d], 0) for d in range(Nd) if (d % 7 == 0 or (d + 1) % 7 == 0)]
)

In [ ]:
c_constraints

[(q_60 == 0, 1), (q_66 == 0, 1), (q_67 == 0, 1), (q_73 == 0, 1), (q_74 == 0, 1), (q_80 == 0, 1), (q_81 == 0, 1), (q_87 == 0, 1), (q_88 == 0, 1)]

## 担当者Dの制約


In [ ]:
# 0-4はpenalty関数で0とする
d_constraints =  sum(
    [penalty(q[3][d]) for d in range(5)]
)

In [ ]:
d_constraints

[(q_90 == 0, 1), (q_91 == 0, 1), (q_92 == 0, 1), (q_93 == 0, 1), (q_94 == 0, 1)]

### イジングマシンの実行

In [ ]:
# 目的関数と制約条件を結合
model = cost + 1 * overwork_constraints +10 * day_constraints + 100 * b_constraints+ 10 * c_constraints + 20 * d_constraints
result = solver.solve(model)
if len(result) == 0:
    raise RuntimeError("条件を満たす解は見つかりませんでした。")
print(f"f={result[0].energy}") # 最適化関数の値

f=192.0


In [ ]:
# 解の数
len(result)

492

In [ ]:
# 解0の結果
r0 = decode_solution(q, result[0].values)
get_calendar(r0)

,1(日),2(月),3(火),4(水),5(木),6(金),7(土),8(日),9(月),10(火),11(水),12(木),13(金),14(土),15(日),16(月),17(火),18(水),19(木),20(金),21(土),22(日),23(月),24(火),25(水),26(木),27(金),28(土),29(日),30(月),出勤日数
A,1,1,1,1,1,0,0,0,1,1,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,1,0,0,0,0,17
B,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,9
C,0,1,1,1,1,1,0,0,0,1,1,0,1,0,0,1,1,1,1,1,0,0,1,0,0,1,1,0,0,1,17
D,0,0,0,0,0,1,1,1,1,0,0,1,1,1,0,0,1,0,0,1,0,1,1,1,1,0,1,1,1,1,17
出勤者数,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,60


In [ ]:
# 解1の結果
r1 = decode_solution(q, result[1].values)
get_calendar(r1)

,1(日),2(月),3(火),4(水),5(木),6(金),7(土),8(日),9(月),10(火),11(水),12(木),13(金),14(土),15(日),16(月),17(火),18(水),19(木),20(金),21(土),22(日),23(月),24(火),25(水),26(木),27(金),28(土),29(日),30(月),出勤日数
A,1,1,1,1,1,0,0,1,0,0,1,1,0,1,1,0,1,0,1,1,1,0,1,0,1,0,1,0,0,0,17
B,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,9
C,0,1,1,1,1,1,0,0,1,1,0,0,1,0,0,1,1,1,1,1,0,0,0,1,0,1,1,0,0,1,17
D,0,0,0,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,0,0,0,1,1,1,1,1,0,1,1,1,17
出勤者数,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,60


## 課題

* 制約にかける係数の値次第で解が見つからない場合があり、適切な係数の値をどう見つけるのかよく分かっていないです。